In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os
import cv2
import splitfolders

In [2]:
data_dir = '../Brain_Projects/brain_tumor_dataset'
splitfolders.ratio(data_dir, output='dataset', seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

Copying files: 253 files [00:01, 148.01 files/s]


In [3]:
IMG_SIZE = 120
BATCH_SIZE = 32

In [4]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [5]:
train_dir = '../Brain_Projects/dataset/train'
test_dir = '../Brain_Projects/dataset/test'
val_dir = '../Brain_Projects/dataset/val'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 202 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Found 27 images belonging to 2 classes.


In [6]:
import tensorflow_hub as hub

Inception_V3 = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5")

In [7]:
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = Inception_V3(inputs)
outputs = layers.Dense(2, activation='softmax')(x)

model_0 = tf.keras.Model(inputs, outputs, name='Model_0_untrainable')

In [8]:
model_0.summary()

Model: "Model_0_untrainable"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 1001)              23853833  
_________________________________________________________________
dense (Dense)                (None, 2)                 2004      
Total params: 23,855,837
Trainable params: 2,004
Non-trainable params: 23,853,833
_________________________________________________________________


In [9]:
model_0.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [10]:
model_0_history = model_0.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

Epoch 1/5
7/7 [==============================] - 29s 864ms/step - loss: 4.5083 - accuracy: 0.5594 - val_loss: 3.3207 - val_accuracy: 0.6250
Epoch 2/5
7/7 [==============================] - 1s 205ms/step - loss: 2.0826 - accuracy: 0.6881 - val_loss: 1.6853 - val_accuracy: 0.8750
Epoch 3/5
7/7 [==============================] - 1s 144ms/step - loss: 1.2221 - accuracy: 0.7723 - val_loss: 1.3518 - val_accuracy: 0.8750
Epoch 4/5
7/7 [==============================] - 1s 126ms/step - loss: 0.8408 - accuracy: 0.7970 - val_loss: 1.7237 - val_accuracy: 0.8333
Epoch 5/5
7/7 [==============================] - 1s 121ms/step - loss: 0.6162 - accuracy: 0.8564 - val_loss: 1.4362 - val_accuracy: 0.8333


In [11]:
model_0.evaluate(test_generator)

1/1 [==============================] - 1s 1s/step - loss: 1.1898 - accuracy: 0.8148


[1.1898244619369507, 0.8148148059844971]

In [12]:
model_0.evaluate(val_generator)

1/1 [==============================] - 0s 178ms/step - loss: 1.4362 - accuracy: 0.8333


[1.4361897706985474, 0.8333333134651184]

In [13]:
Inception_V3_custom = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5")
Inception_V3_custom.trainable = True

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = Inception_V3_custom(inputs)
outputs = layers.Dense(2, activation='softmax')(x)
model_1 = tf.keras.Model(inputs, outputs, name='Model_1_trainable')

model_1.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [14]:
model_1.summary()

Model: "Model_1_trainable"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
keras_layer_1 (KerasLayer)   (None, 1001)              23853833  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2004      
Total params: 23,855,837
Trainable params: 23,821,405
Non-trainable params: 34,432
_________________________________________________________________


In [15]:
model_1_history = model_1.fit(
    train_generator,
    validation_data = val_generator,
    epochs=5
)

Epoch 1/5
7/7 [==============================] - 23s 757ms/step - loss: 1.8301 - accuracy: 0.6287 - val_loss: 154.7309 - val_accuracy: 0.3750
Epoch 2/5
7/7 [==============================] - 2s 254ms/step - loss: 1.3863 - accuracy: 0.8564 - val_loss: 626.2628 - val_accuracy: 0.3750
Epoch 3/5
7/7 [==============================] - 2s 253ms/step - loss: 1.5205 - accuracy: 0.8564 - val_loss: 90.1580 - val_accuracy: 0.6250
Epoch 4/5
7/7 [==============================] - 2s 247ms/step - loss: 1.5532 - accuracy: 0.8119 - val_loss: 3684.5164 - val_accuracy: 0.6250
Epoch 5/5
7/7 [==============================] - 2s 253ms/step - loss: 1.1111 - accuracy: 0.8515 - val_loss: 126725.6016 - val_accuracy: 0.6250


In [16]:
model_1.evaluate(val_generator)

1/1 [==============================] - 0s 162ms/step - loss: 126725.6016 - accuracy: 0.6250


[126725.6015625, 0.625]

In [17]:
model_1.evaluate(test_generator)

1/1 [==============================] - 0s 200ms/step - loss: 110620.1797 - accuracy: 0.5926


[110620.1796875, 0.5925925970077515]

In [50]:
#Model-0
test_generator.reset()
model_0_pred_probs = model_0.predict(test_generator)
model_0_pred_probs

array([[2.80297013e-06, 9.99997139e-01],
       [9.99896288e-01, 1.03756516e-04],
       [1.03214830e-01, 8.96785140e-01],
       [7.15939450e-08, 9.99999881e-01],
       [6.21098101e-01, 3.78901929e-01],
       [5.22324741e-01, 4.77675259e-01],
       [7.70129636e-03, 9.92298663e-01],
       [9.99529481e-01, 4.70515050e-04],
       [9.75353062e-01, 2.46468689e-02],
       [5.17761568e-04, 9.99482214e-01],
       [2.29766569e-03, 9.97702301e-01],
       [9.87352967e-01, 1.26469908e-02],
       [9.99630332e-01, 3.69706278e-04],
       [4.09770869e-02, 9.59022939e-01],
       [1.72925866e-04, 9.99827087e-01],
       [2.61599052e-04, 9.99738395e-01],
       [1.02306463e-01, 8.97693574e-01],
       [1.32217934e-08, 1.00000000e+00],
       [9.22194376e-05, 9.99907732e-01],
       [2.22754466e-06, 9.99997735e-01],
       [9.94641185e-01, 5.35874208e-03],
       [1.00000000e+00, 4.91610530e-10],
       [5.68857729e-01, 4.31142241e-01],
       [1.07380651e-01, 8.92619312e-01],
       [9.821202

In [51]:
model_0_preds = tf.argmax(model_0_pred_probs, axis=1)
model_0_preds

<tf.Tensor: shape=(27,), dtype=int64, numpy=
array([1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0], dtype=int64)>

In [52]:
labels = (train_generator.class_indices)
print(labels)

{'No': 0, 'Yes': 1}


In [53]:
labels = dict((v,k) for k,v in labels.items())
print(labels)

{0: 'No', 1: 'Yes'}


In [54]:
new_model_0_preds = np.array(model_0_preds).tolist()
new_model_0_preds

[1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0]

In [55]:
predictions = [labels[k] for k in new_model_0_preds]
len(predictions)

27

In [60]:
filenames = test_generator.filenames
filenames

['No\\3 no.jpg',
 'No\\49 no.jpg',
 'No\\7 no.jpg',
 'No\\N1.JPG',
 'No\\No15.jpg',
 'No\\No19.jpg',
 'No\\no 4.jpg',
 'No\\no 89.jpg',
 'No\\no 9.png',
 'No\\no 94.jpg',
 'No\\no 96.jpg',
 'Yes\\Y114.JPG',
 'Yes\\Y15.jpg',
 'Yes\\Y166.JPG',
 'Yes\\Y183.jpg',
 'Yes\\Y250.jpg',
 'Yes\\Y257.jpg',
 'Yes\\Y3.jpg',
 'Yes\\Y30.jpg',
 'Yes\\Y35.jpg',
 'Yes\\Y37.jpg',
 'Yes\\Y39.jpg',
 'Yes\\Y45.JPG',
 'Yes\\Y77.jpg',
 'Yes\\Y90.jpg',
 'Yes\\Y92.png',
 'Yes\\Y98.JPG']

In [61]:
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results

,Filename,Predictions
0,No\3 no.jpg,Yes
1,No\49 no.jpg,No
2,No\7 no.jpg,Yes
3,No\N1.JPG,Yes
4,No\No15.jpg,No
5,No\No19.jpg,No
6,No\no 4.jpg,Yes
7,No\no 89.jpg,No
8,No\no 9.png,No
9,No\no 94.jpg,Yes
